In [1]:
import numpy
import random
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import datetime
from datetime import date
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from PIL import Image
import pandas
from sklearn.linear_model import LinearRegression, LogisticRegression
from mpl_toolkits.mplot3d import Axes3D
import seaborn
import scipy
from scipy import stats
from scipy.stats import poisson, ttest_ind
from scipy import linalg
import xarray as xr
import os
import netCDF4
import cftime

In [2]:
Diri = "data/processed/"

In [3]:
# Define Directory
getDiri = "data/"
#
Temp_Files = os.listdir(getDiri+"orig/tas/global/")
print (Temp_Files)

['rlds', 'rlus', 'rsus', 'rsds', 'netTOA']
['tas_CCSM3_control_1530.nc', 'tas_CCSM3II_control_3805.nc', 'tas_CESM104_control_1000.nc', 'tas_FAMOUS_control_3000.nc', 'tas_MPIESM12_abrupt4x_1000.nc', 'tas_GISSE2R_abrupt4x_5001.nc', 'tas_ECHAM5MPIOM_control_100.nc', 'tas_IPSLCM5A_abrupt4x_1000.nc', 'tas_MPIESM12_control_1237.nc', 'tas_CNRMCM61_control_2000.nc', 'tas_CCSM3_abrupt4x_2120.nc', 'tas_HadGEM2_abrupt4x_1299.nc', 'tas_GISSE2R_control_5525.nc', 'tas_IPSLCM5A_control_1000.nc', 'tas_CESM104_abrupt4x_5900.nc', 'tas_MPIESM11_control_2000.nc', 'tas_HadGEM2_control_239.nc', 'tas_CCSM3II_abrupt1400ppm_3132.nc', 'tas_HadCM3L_control_1000.nc', 'tas_ECHAM5MPIOM_abrupt4x_1001.nc', 'tas_FAMOUS_abrupt4x_3000.nc', 'tas_CNRMCM61_abrupt4x_1850.nc', 'tas_HadCM3L_abrupt4x_1000.nc', 'tas_MPIESM11_abrupt4x_4459.nc']


In [5]:
# Get Files
flux_Files = os.listdir(getDiri+"spatial/rsds/") ## These data must be acquired from the LongRunMIP team
print (flux_Files)

['files.txt', 'rsds_ann_HadGEM2_abrupt4x_1299_g025.nc', 'rsds_mon_CNRMCM61_abrupt2x_750_g025.nc', 'rsds_mon_MIROC32_control_680_g025.nc', 'rsds_ann_MPIESMLR_control_2000_g025.nc', 'rsds_mon_ECHAM5MPIOM_abrupt4x_1001_g025.nc', 'rsds_mon_CESM104_abrupt8x_5100_g025.nc', 'rsds_mon_HadCM3L_abrupt4x_1000_g025.nc', 'files.txt~', 'rsds_ann_MPIESMLR_abrupt4x_4459_g025.nc', 'rsds_mon_GISSE2R_abrupt4x_5001_g025.nc', 'rsds_mon_HadCM3L_control_1000_g025.nc', 'rsds_mon_GISSE2R_control_5225_g025.nc', 'rsds_ann_FAMOUS_control_3000_g025.nc', 'rsds_mon_IPSL_control_1000_g025.nc', 'rsds_mon_CCSM3_abrupt4x_2120_g025.nc', 'rsds_mon_CESM104_abrupt4x_5900_g025.nc', 'rsds_mon_MIROC_abrupt4x_150_g025.nc', 'rsds_mon_MPIESM12_abrupt4x_999_g025.nc', 'rsds_mon_MPIESM12_control_1237_g025.nc', 'rsds_mon_CNRMCM61_control_2000_g025.nc', 'rsds_mon_CNRMCM61_abrupt4x_1850_g025.nc', 'rsds_ann_FAMOUS_abrupt4x_3000_g025.nc', 'rsds_mon_ECHAM5MPIOM_control_100_g025.nc', 'rsds_mon_IPSL_abrupt4x_1000_g025.nc', 'rsds_mon_CCSM3_c

In [14]:
# Open Precipitation Files
Count = 0
for File in flux_Files:
    if File[0:9]!="files.txt": 
        Count += 1
        Parts = File.split("_")
        Freq = Parts[1]
        Model = Parts[2]
        Experiment = Parts[3]
        Time = Parts[4]
        print (Freq, Model, Experiment, Time)
print (Count)

ann HadGEM2 abrupt4x 1299
mon CNRMCM61 abrupt2x 750
mon MIROC32 control 680
ann MPIESMLR control 2000
mon ECHAM5MPIOM abrupt4x 1001
mon CESM104 abrupt8x 5100
mon HadCM3L abrupt4x 1000
ann MPIESMLR abrupt4x 4459
mon GISSE2R abrupt4x 5001
mon HadCM3L control 1000
mon GISSE2R control 5225
ann FAMOUS control 3000
mon IPSL control 1000
mon CCSM3 abrupt4x 2120
mon CESM104 abrupt4x 5900
mon MIROC abrupt4x 150
mon MPIESM12 abrupt4x 999
mon MPIESM12 control 1237
mon CNRMCM61 control 2000
mon CNRMCM61 abrupt4x 1850
ann FAMOUS abrupt4x 3000
mon ECHAM5MPIOM control 100
mon IPSL abrupt4x 1000
mon CCSM3 control 1530
mon CESM104 control 1000
ann HadGEM2 control 239
26


In [15]:
# Create List of Models
Model_List = ['MPIESM12', 'GISSE2R', 'IPSLCM5A', 'CCSM3', 'HadGEM2ES', 'CESM104', \
'ECHAM5MPIOM', 'FAMOUS', 'CNRMCM61', 'HadCM3L', 'MPIESM11']

In [16]:
# Open Temperature Files
for File in Temp_Files:
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    Time = Parts[3].split(".")[0]
    if Model not in Model_List:
        Temp_Files.remove(File)
    else:
        if Experiment == "abrupt4x" or Experiment == "control":
            print (Model, Experiment, Time)
        else:
            Temp_Files.remove(File)
print (Temp_Files)

CCSM3 control 1530
FAMOUS control 3000
MPIESM12 abrupt4x 1000
GISSE2R abrupt4x 5001
ECHAM5MPIOM control 100
IPSLCM5A abrupt4x 1000
MPIESM12 control 1237
CNRMCM61 control 2000
CCSM3 abrupt4x 2120
IPSLCM5A control 1000
CESM104 abrupt4x 5900
MPIESM11 control 2000
HadCM3L control 1000
ECHAM5MPIOM abrupt4x 1001
FAMOUS abrupt4x 3000
CNRMCM61 abrupt4x 1850
HadCM3L abrupt4x 1000
MPIESM11 abrupt4x 4459
['tas_CCSM3_control_1530.nc', 'tas_CESM104_control_1000.nc', 'tas_FAMOUS_control_3000.nc', 'tas_MPIESM12_abrupt4x_1000.nc', 'tas_GISSE2R_abrupt4x_5001.nc', 'tas_ECHAM5MPIOM_control_100.nc', 'tas_IPSLCM5A_abrupt4x_1000.nc', 'tas_MPIESM12_control_1237.nc', 'tas_CNRMCM61_control_2000.nc', 'tas_CCSM3_abrupt4x_2120.nc', 'tas_GISSE2R_control_5525.nc', 'tas_IPSLCM5A_control_1000.nc', 'tas_CESM104_abrupt4x_5900.nc', 'tas_MPIESM11_control_2000.nc', 'tas_CCSM3II_abrupt1400ppm_3132.nc', 'tas_HadCM3L_control_1000.nc', 'tas_ECHAM5MPIOM_abrupt4x_1001.nc', 'tas_FAMOUS_abrupt4x_3000.nc', 'tas_CNRMCM61_abrupt4x_1

In [19]:
# Create Lists of Frequency, 4x Time and Control Time
Freq_List = [" ", " ", " ", " ", " ", " ", " ", " ", " ", " ", " "]
Time_4x_List = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Time_Control_List = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#
# Insert Info Into DataFrame
for File in flux_Files:
    if File[0:9]!="files.txt": 
        Parts = File.split("_")
        Freq = Parts[1]
        Model = Parts[2]
        Experiment = Parts[3]
        Time = Parts[4]
        for i in range(len(Model_List)):
            if Model == Model_List[i]:
                Freq_List[i] = Freq
                if Experiment == "abrupt4x":
                    Time_4x_List[i] = Time
                elif Experiment == "control":
                    Time_Control_List[i] = Time
#
# Create DataFrame to Store Info
Model_DF = pandas.DataFrame({"Model": Model_List, "Frequency": Freq_List, \
"Abrupt4x Time": Time_4x_List, "Control Time": Time_Control_List})
Model_DF

,Model,Frequency,Abrupt4x Time,Control Time
0,MPIESM12,mon,999,1237
1,GISSE2R,mon,5001,5225
2,IPSLCM5A,,0,0
3,CCSM3,mon,2120,1530
4,HadGEM2ES,,0,0
5,CESM104,mon,5900,1000
6,ECHAM5MPIOM,mon,1001,100
7,FAMOUS,ann,3000,3000
8,CNRMCM61,mon,1850,2000
9,HadCM3L,mon,1000,1000


In [41]:
# Create List of Models
Model_List = ['MPIESM12', 'GISSE2R',  # 'IPSLCM5A', 
              'CCSM3',    #'HadGEM2ES',
              'CESM104', \
'ECHAM5MPIOM', 'FAMOUS', 'CNRMCM61', 'HadCM3L']#, 'MPIESM11']

In [44]:
len(Model_List)

8

In [45]:
# Create Lists of Frequency, 4x Time and Control Time
Freq_List = [" ", " ", " ", " ", " ", " ", " ", " "]#, " ", " ", " "]
Time_4x_List = [0, 0, 0, 0, 0, 0, 0, 0 ]#, 0, 0, 0]
Time_Control_List = [0, 0, 0, 0, 0, 0, 0, 0]#, 0, 0, 0]
#
# Insert Info Into DataFrame
for File in flux_Files:
    if File[0:9]!="files.txt": 
        Parts = File.split("_")
        Freq = Parts[1]
        Model = Parts[2]
        Experiment = Parts[3]
        Time = Parts[4]
        for i in range(len(Model_List)):
            if Model == Model_List[i]:
                Freq_List[i] = Freq
                if Experiment == "abrupt4x":
                    Time_4x_List[i] = Time
                elif Experiment == "control":
                    Time_Control_List[i] = Time
#
# Create DataFrame to Store Info
Model_DF = pandas.DataFrame({"Model": Model_List, "Frequency": Freq_List, \
"Abrupt4x Time": Time_4x_List, "Control Time": Time_Control_List})
Model_DF

,Model,Frequency,Abrupt4x Time,Control Time
0,MPIESM12,mon,999,1237
1,GISSE2R,mon,5001,5225
2,CCSM3,mon,2120,1530
3,CESM104,mon,5900,1000
4,ECHAM5MPIOM,mon,1001,100
5,FAMOUS,ann,3000,3000
6,CNRMCM61,mon,1850,2000
7,HadCM3L,mon,1000,1000


In [46]:
# Create Precip Control Dictionary
flux_Cont_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    flux_Cont_Dict[Model] = i
    i += 1
flux_Cont_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': [],
 'CCSM3': 2,
 'HadGEM2ES': [],
 'CESM104': 3,
 'ECHAM5MPIOM': 4,
 'FAMOUS': 5,
 'CNRMCM61': 6,
 'HadCM3L': 7,
 'MPIESM11': []}

In [47]:
# Create Anomaly Dictionary
flux_Anom_Dict = {"MPIESM12": [], "GISSE2R": [], "IPSLCM5A": [], "CCSM3": [], "HadGEM2ES": [], "CESM104": [], \
"ECHAM5MPIOM": [], "FAMOUS": [], "CNRMCM61": [], "HadCM3L": [], "MPIESM11": []}
i = 0
for Model in Model_List:
    flux_Anom_Dict[Model] = i
    i += 1
flux_Anom_Dict

{'MPIESM12': 0,
 'GISSE2R': 1,
 'IPSLCM5A': [],
 'CCSM3': 2,
 'HadGEM2ES': [],
 'CESM104': 3,
 'ECHAM5MPIOM': 4,
 'FAMOUS': 5,
 'CNRMCM61': 6,
 'HadCM3L': 7,
 'MPIESM11': []}

In [48]:
var = "rsds"

In [29]:
%%time
# Open flux Datasets
i = 0
for Model in Model_List:
    print (Model)
# Find Filenames
    flux_4x_Filename = var+"_"+Freq_List[i]+"_"+Model+"_"+"abrupt4x"+"_"+Time_4x_List[i]+"_g025.nc"
    flux_Control_Filename = var+"_"+Freq_List[i]+"_"+Model+"_"+"control"+"_"+Time_Control_List[i]+"_g025.nc"
#
# Open 4x and Control Datasets 0-999 Years
    if Model == "MPIESM12" or Model == "IPSLCM5A" or Model == "CCSM3" or Model == "HadGEM2ES":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "GISSE2R":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(1850,1,1), cftime.DatetimeNoLeap(2849,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(3981,1,1), cftime.DatetimeNoLeap(4980,12,31)))
    if Model == "CESM104":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(420,1,1), cftime.DatetimeNoLeap(1419,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "FAMOUS":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
    if Model == "CNRMCM61":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    if Model == "ECHAM5MPIOM":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time))
    if Model == "HadCM3L":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_4x_New_Time.append((k + 0.5) / 12)
        flux_4x_Data = flux_4x_Data.assign_coords(month=("month", flux_4x_New_Time))
        flux_Control_New_Time = (numpy.array(flux_Control_Data.month) - 0.7) / 12
        flux_Control_Data = flux_Control_Data.assign_coords(month=("month", flux_Control_New_Time))
    if Model == "MPIESM11":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    print ("Step A")
#
# Calculate Annual Abrupt4x and Control Data
    if Model != "HadCM3L":
        flux_4x_Annual = flux_4x_Data.groupby('time.year').mean(dim='time')
        flux_Control_Annual = flux_Control_Data.groupby('time.year').mean(dim='time')
    else:
        Month_Bins = numpy.arange(0,int(Time_4x_List[i])+1,1)
        flux_4x_Bins = flux_4x_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_4x_Years = []
        for k in range(len(flux_4x_Bins)):
            flux_4x_Years.append(k)
        flux_4x_Annual = flux_4x_Bins.assign_coords(month_bins=("month_bins", flux_4x_Years))
        flux_4x_Annual = flux_4x_Annual.rename({"month_bins": "year"})
        flux_Control_Bins = flux_Control_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_Control_Years = []
        for k in range(len(flux_Control_Bins)):
            flux_Control_Years.append(k)
        flux_Control_Annual = flux_Control_Bins.assign_coords(month_bins=("month_bins", flux_Control_Years))
        flux_Control_Annual = flux_Control_Annual.rename({"month_bins": "year"})
    print ("Step B")
    if Model != "HadCM3L":
        try:
            flux_Control_Mean = flux_Control_Data.mean(dim='time') + flux_4x_Annual * 0 
        except:
            flux_Control_Mean = flux_Control_Data.mean(dim='year') + flux_4x_Annual * 0
    else:
        flux_Control_Mean = flux_Control_Data.mean(dim='month') + flux_4x_Annual * 0
    print ("Step C")
#
# Edit Year to Start at 0 for Abrupt 4x and Control Data
    if Model == "MPIESM12" or Model == "CNRMCM61" or Model == "ECHAM5MPIOM" or Model == "MPIESM11":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1850
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "IPSLCM5A":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1800
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "GISSE2R":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 3981
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "CCSM3":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1
    if Model == "HadGEM2ES":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1860
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1860
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1860
    if Model == "CESM104":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 420
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 420
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 420
    if Model == "FAMOUS":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 5002
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 5002
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 5002
    print ("Step D")
#
# Calculate Precip Anomaly
    flux_Control = flux_Control_Annual.sel(year=slice(0,999))
    flux_Anomaly = flux_4x_Annual.sel(year=slice(0,999)) - flux_Control_Mean.sel(year=slice(0,999))
    print ("Step E")
#
#
# Create New Net CDF Files
    Filename_1 = Diri+"Net_CDF/"+var+"_Anom_"+Model+".nc"
    flux_Anomaly.to_netcdf(path=Filename_1)
    Filename_2 = Diri+"Net_CDF/"+var+"_Control_"+Model+".nc"
    flux_Control.to_netcdf(path=Filename_2)
    print ("Step F")
    i += 1

MPIESM12
Step A
Step B
Step C
Step D
Step E
Step F
GISSE2R
Step A
Step B
Step C
Step D
Step E
Step F
IPSLCM5A


TypeError: can only concatenate str (not "int") to str

In [49]:
%%time
# Open flux Datasets
i = 2
for Model in Model_List[2:]:
    print (Model)
# Find Filenames
    flux_4x_Filename = var+"_"+Freq_List[i]+"_"+Model+"_"+"abrupt4x"+"_"+Time_4x_List[i]+"_g025.nc"
    flux_Control_Filename = var+"_"+Freq_List[i]+"_"+Model+"_"+"control"+"_"+Time_Control_List[i]+"_g025.nc"
#
# Open 4x and Control Datasets 0-999 Years
    if Model == "MPIESM12" or Model == "IPSLCM5A" or Model == "CCSM3" or Model == "HadGEM2ES":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "GISSE2R":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(1850,1,1), cftime.DatetimeNoLeap(2849,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(3981,1,1), cftime.DatetimeNoLeap(4980,12,31)))
    if Model == "CESM104":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(420,1,1), cftime.DatetimeNoLeap(1419,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "FAMOUS":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
    if Model == "CNRMCM61":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    if Model == "ECHAM5MPIOM":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time))
    if Model == "HadCM3L":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_4x_New_Time.append((k + 0.5) / 12)
        flux_4x_Data = flux_4x_Data.assign_coords(month=("month", flux_4x_New_Time))
        flux_Control_New_Time = (numpy.array(flux_Control_Data.month) - 0.7) / 12
        flux_Control_Data = flux_Control_Data.assign_coords(month=("month", flux_Control_New_Time))
    if Model == "MPIESM11":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    print ("Step A")
#
# Calculate Annual Abrupt4x and Control Data
    if Model != "HadCM3L":
        flux_4x_Annual = flux_4x_Data.groupby('time.year').mean(dim='time')
        flux_Control_Annual = flux_Control_Data.groupby('time.year').mean(dim='time')
    else:
        Month_Bins = numpy.arange(0,int(Time_4x_List[i])+1,1)
        flux_4x_Bins = flux_4x_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_4x_Years = []
        for k in range(len(flux_4x_Bins)):
            flux_4x_Years.append(k)
        flux_4x_Annual = flux_4x_Bins.assign_coords(month_bins=("month_bins", flux_4x_Years))
        flux_4x_Annual = flux_4x_Annual.rename({"month_bins": "year"})
        flux_Control_Bins = flux_Control_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_Control_Years = []
        for k in range(len(flux_Control_Bins)):
            flux_Control_Years.append(k)
        flux_Control_Annual = flux_Control_Bins.assign_coords(month_bins=("month_bins", flux_Control_Years))
        flux_Control_Annual = flux_Control_Annual.rename({"month_bins": "year"})
    print ("Step B")
    if Model != "HadCM3L":
        try:
            flux_Control_Mean = flux_Control_Data.mean(dim='time') + flux_4x_Annual * 0 
        except:
            flux_Control_Mean = flux_Control_Data.mean(dim='year') + flux_4x_Annual * 0
    else:
        flux_Control_Mean = flux_Control_Data.mean(dim='month') + flux_4x_Annual * 0
    print ("Step C")
#
# Edit Year to Start at 0 for Abrupt 4x and Control Data
    if Model == "MPIESM12" or Model == "CNRMCM61" or Model == "ECHAM5MPIOM" or Model == "MPIESM11":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1850
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "IPSLCM5A":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1800
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "GISSE2R":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 3981
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "CCSM3":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1
    if Model == "HadGEM2ES":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1860
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1860
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1860
    if Model == "CESM104":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 420
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 420
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 420
    if Model == "FAMOUS":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 5002
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 5002
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 5002
    print ("Step D")
#
# Calculate Precip Anomaly
    flux_Control = flux_Control_Annual.sel(year=slice(0,999))
    flux_Anomaly = flux_4x_Annual.sel(year=slice(0,999)) - flux_Control_Mean.sel(year=slice(0,999))
    print ("Step E")
#
#
# Create New Net CDF Files
    Filename_1 = Diri+"Net_CDF/"+var+"_Anom_"+Model+".nc"
    flux_Anomaly.to_netcdf(path=Filename_1)
    Filename_2 = Diri+"Net_CDF/"+var+"_Control_"+Model+".nc"
    flux_Control.to_netcdf(path=Filename_2)
    print ("Step F")
    i += 1

CCSM3
Step A
Step B
Step C
Step D
Step E
Step F
CESM104
Step A
Step B
Step C
Step D
Step E
Step F
ECHAM5MPIOM
Step A
Step B
Step C
Step D
Step E
Step F
FAMOUS
Step A
Step B
Step C
Step D
Step E
Step F
CNRMCM61
Step A
Step B
Step C
Step D
Step E
Step F
HadCM3L
Step A
Step B
Step C
Step D
Step E
Step F
CPU times: user 26.4 s, sys: 10 s, total: 36.4 s
Wall time: 44.7 s


In [ ]:
#
# Open 4x and Control Datasets 0-999 Years
    if Model == "MPIESM12" or Model == "IPSLCM5A" or Model == "CCSM3" or Model == "HadGEM2ES":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "GISSE2R":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(1850,1,1), cftime.DatetimeNoLeap(2849,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(3981,1,1), cftime.DatetimeNoLeap(4980,12,31)))
    if Model == "CESM104":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeNoLeap(420,1,1), cftime.DatetimeNoLeap(1419,12,31)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
    if Model == "FAMOUS":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.Datetime360Day(5002,1,1), cftime.Datetime360Day(6001,12,1)))
    if Model == "CNRMCM61":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var].sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    if Model == "ECHAM5MPIOM":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            int(1850 + numpy.floor(k/12)), (k%12) + 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time))
    if Model == "HadCM3L":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_4x_New_Time.append((k + 0.5) / 12)
        flux_4x_Data = flux_4x_Data.assign_coords(month=("month", flux_4x_New_Time))
        flux_Control_New_Time = (numpy.array(flux_Control_Data.month) - 0.7) / 12
        flux_Control_Data = flux_Control_Data.assign_coords(month=("month", flux_Control_New_Time))
    if Model == "MPIESM11":
        flux_4x_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_4x_Filename, use_cftime=True)[var]
        flux_Control_Data = xr.open_dataset(getDiri+"spatial/"+var+"/"+flux_Control_Filename, use_cftime=True)[var]
        flux_4x_New_Time = []
        for k in range(len(flux_4x_Data)):
            flux_4x_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_4x_Data = flux_4x_Data.assign_coords(time=("time", flux_4x_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
        flux_Control_New_Time = []
        for k in range(len(flux_Control_Data)):
            flux_Control_New_Time.append(cftime.DatetimeProlepticGregorian(\
            1850 + k, 1, 1, 7, 28, 0, 0, has_year_zero=True))
        flux_Control_Data = flux_Control_Data.assign_coords(time=("time", flux_Control_New_Time)).sel\
        (time=slice(cftime.DatetimeGregorian(1850,1,1), cftime.DatetimeGregorian(2849,12,1)))
    print ("Step A")
#
# Calculate Annual Abrupt4x and Control Data
    if Model != "HadCM3L":
        flux_4x_Annual = flux_4x_Data.groupby('time.year').mean(dim='time')
        flux_Control_Annual = flux_Control_Data.groupby('time.year').mean(dim='time')
    else:
        Month_Bins = numpy.arange(0,int(Time_4x_List[i])+1,1)
        flux_4x_Bins = flux_4x_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_4x_Years = []
        for k in range(len(flux_4x_Bins)):
            flux_4x_Years.append(k)
        flux_4x_Annual = flux_4x_Bins.assign_coords(month_bins=("month_bins", flux_4x_Years))
        flux_4x_Annual = flux_4x_Annual.rename({"month_bins": "year"})
        flux_Control_Bins = flux_Control_Data.groupby_bins('month', Month_Bins).mean("month")
        flux_Control_Years = []
        for k in range(len(flux_Control_Bins)):
            flux_Control_Years.append(k)
        flux_Control_Annual = flux_Control_Bins.assign_coords(month_bins=("month_bins", flux_Control_Years))
        flux_Control_Annual = flux_Control_Annual.rename({"month_bins": "year"})
    print ("Step B")
    if Model != "HadCM3L":
        try:
            flux_Control_Mean = flux_Control_Data.mean(dim='time') + flux_4x_Annual * 0 
        except:
            flux_Control_Mean = flux_Control_Data.mean(dim='year') + flux_4x_Annual * 0
    else:
        flux_Control_Mean = flux_Control_Data.mean(dim='month') + flux_4x_Annual * 0
    print ("Step C")
#
# Edit Year to Start at 0 for Abrupt 4x and Control Data
    if Model == "MPIESM12" or Model == "CNRMCM61" or Model == "ECHAM5MPIOM" or Model == "MPIESM11":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1850
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "IPSLCM5A":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1800
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "GISSE2R":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1850
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 3981
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1850
    if Model == "CCSM3":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1
    if Model == "HadGEM2ES":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 1860
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 1860
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 1860
    if Model == "CESM104":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 420
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 420
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 420
    if Model == "FAMOUS":
        flux_4x_Annual['year'] = flux_4x_Annual['year'] - 5002
        flux_Control_Annual['year'] = flux_Control_Annual['year'] - 5002
        flux_Control_Mean['year'] = flux_Control_Mean['year'] - 5002
    print ("Step D")
#
# Calculate Precip Anomaly
    flux_Control = flux_Control_Annual.sel(year=slice(0,999))
    flux_Anomaly = flux_4x_Annual.sel(year=slice(0,999)) - flux_Control_Mean.sel(year=slice(0,999))
    print ("Step E")
#
#
# Create New Net CDF Files
    Filename_1 = Diri+"Net_CDF/"+var+"_Anom_"+Model+".nc"
    flux_Anomaly.to_netcdf(path=Filename_1)
    Filename_2 = Diri+"Net_CDF/"+var+"_Control_"+Model+".nc"
    flux_Control.to_netcdf(path=Filename_2)
    print ("Step F")